<a href="https://colab.research.google.com/github/lamarock61/hello-world/blob/master/Kraken_bot_ksm_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# pip install SimpleJSON;

Import library

In [2]:
#import
import pandas as pd
import requests
import json
import time
import os
import urllib.parse
import hashlib
import hmac
import base64
import json
import math



In [12]:
# initial paremeters
min_buy_price = 8.01;
amount_to_buy = 1;
max_ksm_amount_to_sell = 0.10;
max_ksm_amount_to_buy = 0.05;
min_ksm_amount_in_folio = 0.01;
min_dot_amount_in_folio = 0.01;
dot_amount_in_folio = 0.0000;
ksm_amount_in_folio = 0.0000;
delta = 0.09;
max_sell_price = round(min_buy_price*(1+delta),2);
trade_fees_percentage = 0.001;
print("min buy price : " + str(min_buy_price));
print("min amount to buy : " + str(amount_to_buy));
print("max sell price : " + str(max_sell_price));
print("trading fees : " + str(trade_fees_percentage*100.000) + "%");
API_key = "xxx";
Private_key = "xxx";

min buy price : 8.01
min amount to buy : 1
max sell price : 8.73
trading fees : 0.1%


In [13]:
# dynamic parameters
current_ksm_dpt_price = 0;
dot_amount_in_folio = 0;
ksm_amount_in_folio = 0.2379257;

In [14]:
## get ksm/dot value from kraken
# a = ask
# b = bid
# c = last trade closed
# v = volume
# p = volume weighted average price
# t = number of trades
# l = low
# h = high
# o = today's opening price

resp = requests.get('https://api.kraken.com/0/public/Ticker?pair=KSMDOT')

#print(resp.json())


jsonResponse_current_ksm_dot_price = resp.json();

current_ksm_dot_price = jsonResponse_current_ksm_dot_price["result"]["KSMDOT"]["c"][0];

print("current ksm/dot price : " + current_ksm_dot_price);
print(jsonResponse_current_ksm_dot_price);

current ksm/dot price : 8.56000
{'error': [], 'result': {'KSMDOT': {'a': ['8.58000', '11', '11.000'], 'b': ['8.56000', '9', '9.000'], 'c': ['8.56000', '0.09932962'], 'v': ['1603.02226109', '1666.74682134'], 'p': ['8.77235', '8.77028'], 't': [845, 881], 'l': ['8.36000', '8.36000'], 'h': ['9.07000', '9.07000'], 'o': '8.70000'}}}


In [15]:
print(jsonResponse_current_ksm_dot_price["result"]["KSMDOT"]['c'])

['8.56000', '0.09932962']


In [16]:
# authenticate to kraken

def get_kraken_signature(urlpath, data, secret):

    postdata = urllib.parse.urlencode(data)
    encoded = (str(data['nonce']) + postdata).encode()
    message = urlpath.encode() + hashlib.sha256(encoded).digest()

    mac = hmac.new(base64.b64decode(secret), message, hashlib.sha512)
    sigdigest = base64.b64encode(mac.digest())
    return sigdigest.decode()

In [19]:
# get ksm & dot amount in folio

# Read Kraken API key and secret stored in environment variables
api_url = "https://api.kraken.com"
#api_key = os.environ['API_KEY_KRAKEN']
api_key = API_key;
#api_sec = os.environ['API_SEC_KRAKEN']
api_sec = Private_key

# Attaches auth headers and returns results of a POST request
def kraken_request(uri_path, data, api_key, api_sec):
    headers = {}
    headers['API-Key'] = api_key
    # get_kraken_signature() as defined in the 'Authentication' section
    headers['API-Sign'] = get_kraken_signature(uri_path, data, api_sec)             
    req = requests.post((api_url + uri_path), headers=headers, data=data)
    return req

# Construct the request and print the result
resp = kraken_request('/0/private/Balance', {
    "nonce": str(int(1000*time.time()))
}, api_key, api_sec)

#print(resp.json())


jsonResponse_folio_data = resp.json();

try :
  dot_amount_in_folio = float(jsonResponse_folio_data["result"]["DOT"])
except :
  dot_amount_in_folio = 0.0000;

try :
  ksm_amount_in_folio = float(jsonResponse_folio_data["result"]["KSM"])
except :
  ksm_amount_in_folio = 0.0000;

print(ksm_amount_in_folio)
print(dot_amount_in_folio)



0.0
0.25030934


In [24]:
# get list of open orders
resp = kraken_request('/0/private/OpenOrders', {
    "nonce": str(int(1000*time.time())),
    "trades": True
}, api_key, api_sec)

jsonResponse_open_orders = resp.json();

# get list of open order keys and count keys
list_keys = [];
list_keys = list(jsonResponse_open_orders["result"]["open"].keys())
keys_qty = len(list_keys)
print(keys_qty)


print(jsonResponse_open_orders["result"]["open"]);
# print(list(jsonResponse_open_orders["result"]["open"].keys()));
# print(jsonResponse_open_orders["result"]["open"]["ON66J4-FNUI2-4BXA3R"]["descr"]["pair"]);

if keys_qty == 0:
  sell_orders_qty = 0
  buy_orders_qty = 0

open_order = jsonResponse_open_orders["result"]["open"]
for _key in list_keys:
  if open_order[_key]["descr"]["pair"] == "KSMDOT" and open_order[_key]["descr"]["type"] == "sell":
    sell_orders_qty = 1
    # sell order allready exists
  else:
    sell_orders_qty = 0

for _key in list_keys:
  if open_order[_key]["descr"]["pair"] == "KSMDOT" and open_order[_key]["descr"]["type"] == "buy":
    buy_orders_qty = 1
    # buy order already exists
  else:
    buy_orders_qty = 0

# create sell order if needed
if sell_orders_qty == 0 and dot_amount_in_folio < min_dot_amount_in_folio:
        # place new sell order
  print("place new sell order")
  resp = kraken_request('/0/private/AddOrder', {
      "nonce": str(int(1000*time.time())),
      "ordertype": "limit",
      "type": "sell",
      "volume": min(ksm_amount_in_folio, max_ksm_amount_to_sell),
      "pair": "KSMDOT",
      "price": max_sell_price
  }, api_key, api_sec)

  print(resp.json())
  print("sell order sent")

else:
  print("no sell opportunity")

# create buy order if needed
if buy_orders_qty == 0 and ksm_amount_in_folio < min_ksm_amount_in_folio:
  # place new buy order
  print("place new buy order")
  resp = kraken_request('/0/private/AddOrder', {
      "nonce": str(int(1000*time.time())),
      "ordertype": "limit",
      "type": "buy",
      "volume": min(dot_amount_in_folio/min_buy_price, max_ksm_amount_to_buy),
      "pair": "KSMDOT",
      "price": min_buy_price
  }, api_key, api_sec)

  print(resp.json())
  print("buy order sent")
else:
  print("no buy opportunity")

'''
    if dot_amount_in_folio < min_dot_amount_in_folio:
      print("place new buy order")
      # place new buy order
'''
#print(jsonResponse_open_orders["result"]["open"][_key]["descr"])
print("sell order qty : " + str(sell_orders_qty))
print("buy order qty : " + str(buy_orders_qty))
print(dot_amount_in_folio)
print(max_ksm_amount_to_sell)
print(min_buy_price)

0
{}
no sell opportunity
place new buy order
{'error': [], 'result': {'txid': ['ODAR4T-5JJWF-36KAXP'], 'descr': {'order': 'buy 0.03124960 KSMDOT @ limit 8.01'}}}
buy order sent
sell order qty : 0
buy order qty : 0
0.25030934
0.1
8.01
